In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.7002 eloss:2.9667 aloss2:1.9316 exploreP:0.9984
Episode:1 meanR:19.0000 R:22.0000 rate:0.0440 aloss:0.6945 eloss:2.9355 aloss2:1.9303 exploreP:0.9962
Episode:2 meanR:21.6667 R:27.0000 rate:0.0540 aloss:0.6941 eloss:2.9423 aloss2:1.9285 exploreP:0.9936
Episode:3 meanR:21.7500 R:22.0000 rate:0.0440 aloss:0.6937 eloss:2.9429 aloss2:1.9317 exploreP:0.9914
Episode:4 meanR:22.6000 R:26.0000 rate:0.0520 aloss:0.6929 eloss:2.9322 aloss2:1.9387 exploreP:0.9889
Episode:5 meanR:21.1667 R:14.0000 rate:0.0280 aloss:0.6890 eloss:2.9338 aloss2:1.9213 exploreP:0.9875
Episode:6 meanR:20.7143 R:18.0000 rate:0.0360 aloss:0.6941 eloss:2.9046 aloss2:1.9534 exploreP:0.9857
Episode:7 meanR:19.7500 R:13.0000 rate:0.0260 aloss:0.6924 eloss:3.0231 aloss2:1.8666 exploreP:0.9845
Episode:8 meanR:21.4444 R:35.0000 rate:0.0700 aloss:0.6907 eloss:2.9511 aloss2:1.9179 exploreP:0.9811
Episode:9 meanR:21.3000 R:20.0000 rate:0.0400 aloss:0.6897 eloss:2.9134 aloss2:1.9

Episode:80 meanR:21.6420 R:23.0000 rate:0.0460 aloss:0.7317 eloss:1.9860 aloss2:3.1822 exploreP:0.8408
Episode:81 meanR:21.6463 R:22.0000 rate:0.0440 aloss:0.7236 eloss:1.9930 aloss2:3.1682 exploreP:0.8390
Episode:82 meanR:21.7590 R:31.0000 rate:0.0620 aloss:0.7122 eloss:1.9857 aloss2:3.1567 exploreP:0.8364
Episode:83 meanR:21.7262 R:19.0000 rate:0.0380 aloss:0.7065 eloss:1.9894 aloss2:3.1685 exploreP:0.8349
Episode:84 meanR:21.7647 R:25.0000 rate:0.0500 aloss:0.7080 eloss:1.9817 aloss2:3.1606 exploreP:0.8328
Episode:85 meanR:21.7209 R:18.0000 rate:0.0360 aloss:0.7203 eloss:1.9579 aloss2:3.1869 exploreP:0.8313
Episode:86 meanR:21.6322 R:14.0000 rate:0.0280 aloss:0.7657 eloss:1.9940 aloss2:3.1192 exploreP:0.8302
Episode:87 meanR:21.8295 R:39.0000 rate:0.0780 aloss:0.6946 eloss:1.9760 aloss2:3.2212 exploreP:0.8270
Episode:88 meanR:21.8764 R:26.0000 rate:0.0520 aloss:0.7080 eloss:1.9973 aloss2:3.1234 exploreP:0.8249
Episode:89 meanR:21.8111 R:16.0000 rate:0.0320 aloss:0.6909 eloss:1.9672 

Episode:160 meanR:19.7300 R:19.0000 rate:0.0380 aloss:0.7129 eloss:1.6475 aloss2:4.2974 exploreP:0.7241
Episode:161 meanR:19.5000 R:11.0000 rate:0.0220 aloss:0.7162 eloss:1.6354 aloss2:4.3188 exploreP:0.7233
Episode:162 meanR:19.4400 R:22.0000 rate:0.0440 aloss:0.7119 eloss:1.6025 aloss2:4.3596 exploreP:0.7217
Episode:163 meanR:19.3200 R:18.0000 rate:0.0360 aloss:0.7269 eloss:1.6129 aloss2:4.2934 exploreP:0.7205
Episode:164 meanR:19.5300 R:35.0000 rate:0.0700 aloss:0.7116 eloss:1.5952 aloss2:4.3147 exploreP:0.7180
Episode:165 meanR:19.4000 R:10.0000 rate:0.0200 aloss:0.7139 eloss:1.6046 aloss2:4.7308 exploreP:0.7173
Episode:166 meanR:19.2900 R:10.0000 rate:0.0200 aloss:0.6892 eloss:1.5931 aloss2:4.5366 exploreP:0.7166
Episode:167 meanR:19.4300 R:29.0000 rate:0.0580 aloss:0.7444 eloss:1.6287 aloss2:4.7710 exploreP:0.7145
Episode:168 meanR:19.3100 R:10.0000 rate:0.0200 aloss:0.6932 eloss:1.6313 aloss2:4.3181 exploreP:0.7138
Episode:169 meanR:19.2400 R:16.0000 rate:0.0320 aloss:0.6871 elo

Episode:239 meanR:16.7100 R:23.0000 rate:0.0460 aloss:0.7276 eloss:1.4897 aloss2:5.5645 exploreP:0.6393
Episode:240 meanR:16.6200 R:16.0000 rate:0.0320 aloss:0.7493 eloss:1.4851 aloss2:5.4078 exploreP:0.6383
Episode:241 meanR:16.6700 R:23.0000 rate:0.0460 aloss:0.7618 eloss:1.4535 aloss2:5.3238 exploreP:0.6368
Episode:242 meanR:16.6500 R:15.0000 rate:0.0300 aloss:0.7239 eloss:1.5081 aloss2:5.6339 exploreP:0.6359
Episode:243 meanR:16.6300 R:16.0000 rate:0.0320 aloss:0.7445 eloss:1.4890 aloss2:5.6615 exploreP:0.6349
Episode:244 meanR:16.6400 R:17.0000 rate:0.0340 aloss:0.7271 eloss:1.4952 aloss2:5.2423 exploreP:0.6338
Episode:245 meanR:16.6100 R:10.0000 rate:0.0200 aloss:0.6986 eloss:1.4423 aloss2:5.2537 exploreP:0.6332
Episode:246 meanR:16.5400 R:10.0000 rate:0.0200 aloss:0.7220 eloss:1.5069 aloss2:5.4431 exploreP:0.6326
Episode:247 meanR:16.6100 R:18.0000 rate:0.0360 aloss:0.7261 eloss:1.4716 aloss2:5.5454 exploreP:0.6315
Episode:248 meanR:16.3300 R:18.0000 rate:0.0360 aloss:0.7515 elo

Episode:318 meanR:33.5400 R:35.0000 rate:0.0700 aloss:0.7056 eloss:1.4463 aloss2:6.0223 exploreP:0.4751
Episode:319 meanR:33.9800 R:54.0000 rate:0.1080 aloss:0.7044 eloss:1.4416 aloss2:6.0698 exploreP:0.4726
Episode:320 meanR:34.4800 R:59.0000 rate:0.1180 aloss:0.7132 eloss:1.4199 aloss2:6.1708 exploreP:0.4699
Episode:321 meanR:34.5700 R:25.0000 rate:0.0500 aloss:0.6920 eloss:1.4474 aloss2:6.1602 exploreP:0.4687
Episode:322 meanR:35.2400 R:89.0000 rate:0.1780 aloss:0.7075 eloss:1.4186 aloss2:6.1398 exploreP:0.4647
Episode:323 meanR:35.9500 R:80.0000 rate:0.1600 aloss:0.7057 eloss:1.4053 aloss2:6.2405 exploreP:0.4611
Episode:324 meanR:36.1600 R:38.0000 rate:0.0760 aloss:0.7365 eloss:1.3859 aloss2:6.1882 exploreP:0.4594
Episode:325 meanR:37.3600 R:130.0000 rate:0.2600 aloss:0.7034 eloss:1.3922 aloss2:6.2216 exploreP:0.4535
Episode:326 meanR:37.4800 R:23.0000 rate:0.0460 aloss:0.7164 eloss:1.3992 aloss2:6.2895 exploreP:0.4525
Episode:327 meanR:37.7600 R:51.0000 rate:0.1020 aloss:0.6979 el

...

Episode:2280 meanR:58.6400 R:49.0000 rate:0.0980 aloss:0.3518 eloss:1.6940 aloss2:5.4410 exploreP:0.0100
Episode:2281 meanR:58.9100 R:67.0000 rate:0.1340 aloss:0.3558 eloss:1.5699 aloss2:5.4413 exploreP:0.0100
Episode:2282 meanR:58.8500 R:43.0000 rate:0.0860 aloss:0.3240 eloss:1.6468 aloss2:5.4565 exploreP:0.0100
Episode:2283 meanR:58.8600 R:63.0000 rate:0.1260 aloss:0.3228 eloss:1.6836 aloss2:5.4135 exploreP:0.0100
Episode:2284 meanR:58.7300 R:38.0000 rate:0.0760 aloss:0.3198 eloss:1.6324 aloss2:5.3957 exploreP:0.0100
Episode:2285 meanR:58.6600 R:43.0000 rate:0.0860 aloss:0.3508 eloss:1.7042 aloss2:5.4025 exploreP:0.0100
Episode:2286 meanR:58.8500 R:62.0000 rate:0.1240 aloss:0.3362 eloss:1.6439 aloss2:5.4110 exploreP:0.0100
Episode:2287 meanR:59.1300 R:68.0000 rate:0.1360 aloss:0.3438 eloss:1.6764 aloss2:5.3814 exploreP:0.0100
Episode:2288 meanR:59.5800 R:89.0000 rate:0.1780 aloss:0.3631 eloss:1.6375 aloss2:5.3640 exploreP:0.0100
Episode:2289 meanR:59.5800 R:34.0000 rate:0.0680 aloss:

Episode:2359 meanR:51.1200 R:56.0000 rate:0.1120 aloss:0.3215 eloss:1.6087 aloss2:5.3542 exploreP:0.0100
Episode:2360 meanR:50.9800 R:33.0000 rate:0.0660 aloss:0.3226 eloss:1.6666 aloss2:5.3608 exploreP:0.0100
Episode:2361 meanR:50.8700 R:38.0000 rate:0.0760 aloss:0.3232 eloss:1.6095 aloss2:5.3701 exploreP:0.0100
Episode:2362 meanR:51.1000 R:61.0000 rate:0.1220 aloss:0.3121 eloss:1.6291 aloss2:5.3630 exploreP:0.0100
Episode:2363 meanR:51.4500 R:91.0000 rate:0.1820 aloss:0.3627 eloss:1.6890 aloss2:5.3351 exploreP:0.0100
Episode:2364 meanR:51.3700 R:35.0000 rate:0.0700 aloss:0.3304 eloss:1.6935 aloss2:5.3150 exploreP:0.0100
Episode:2365 meanR:50.8700 R:34.0000 rate:0.0680 aloss:0.3774 eloss:1.6774 aloss2:5.3182 exploreP:0.0100
Episode:2366 meanR:50.9200 R:37.0000 rate:0.0740 aloss:0.3840 eloss:1.5969 aloss2:5.3521 exploreP:0.0100
Episode:2367 meanR:51.0700 R:52.0000 rate:0.1040 aloss:0.3470 eloss:1.6637 aloss2:5.3782 exploreP:0.0100
Episode:2368 meanR:50.8900 R:29.0000 rate:0.0580 aloss:

Episode:2438 meanR:44.3400 R:54.0000 rate:0.1080 aloss:0.3492 eloss:1.7092 aloss2:5.3674 exploreP:0.0100
Episode:2439 meanR:44.2200 R:30.0000 rate:0.0600 aloss:0.3286 eloss:1.5659 aloss2:5.3550 exploreP:0.0100
Episode:2440 meanR:44.4700 R:59.0000 rate:0.1180 aloss:0.3749 eloss:1.7240 aloss2:5.3748 exploreP:0.0100
Episode:2441 meanR:44.6000 R:50.0000 rate:0.1000 aloss:0.3515 eloss:1.6430 aloss2:5.3749 exploreP:0.0100
Episode:2442 meanR:44.4800 R:32.0000 rate:0.0640 aloss:0.3323 eloss:1.6561 aloss2:5.3688 exploreP:0.0100
Episode:2443 meanR:44.6000 R:42.0000 rate:0.0840 aloss:0.3536 eloss:1.6888 aloss2:5.3742 exploreP:0.0100
Episode:2444 meanR:44.9600 R:75.0000 rate:0.1500 aloss:0.3398 eloss:1.6871 aloss2:5.3620 exploreP:0.0100
Episode:2445 meanR:44.9200 R:41.0000 rate:0.0820 aloss:0.3507 eloss:1.7116 aloss2:5.2940 exploreP:0.0100
Episode:2446 meanR:45.3200 R:85.0000 rate:0.1700 aloss:0.3323 eloss:1.6185 aloss2:5.3089 exploreP:0.0100
Episode:2447 meanR:45.2800 R:46.0000 rate:0.0920 aloss:

Episode:2517 meanR:44.4000 R:29.0000 rate:0.0580 aloss:0.2964 eloss:1.5702 aloss2:5.4442 exploreP:0.0100
Episode:2518 meanR:44.3400 R:48.0000 rate:0.0960 aloss:0.2695 eloss:1.5429 aloss2:5.4822 exploreP:0.0100
Episode:2519 meanR:44.4100 R:40.0000 rate:0.0800 aloss:0.3305 eloss:1.6035 aloss2:5.5271 exploreP:0.0100
Episode:2520 meanR:44.0300 R:32.0000 rate:0.0640 aloss:0.2970 eloss:1.6921 aloss2:5.5194 exploreP:0.0100
Episode:2521 meanR:43.9700 R:36.0000 rate:0.0720 aloss:0.2736 eloss:1.6331 aloss2:5.4725 exploreP:0.0100
Episode:2522 meanR:43.9600 R:51.0000 rate:0.1020 aloss:0.2738 eloss:1.6693 aloss2:5.4990 exploreP:0.0100
Episode:2523 meanR:43.9800 R:34.0000 rate:0.0680 aloss:0.2914 eloss:1.5951 aloss2:5.4628 exploreP:0.0100
Episode:2524 meanR:44.3100 R:66.0000 rate:0.1320 aloss:0.2987 eloss:1.6358 aloss2:5.4385 exploreP:0.0100
Episode:2525 meanR:44.1700 R:26.0000 rate:0.0520 aloss:0.3154 eloss:1.6039 aloss2:5.4451 exploreP:0.0100
Episode:2526 meanR:43.9900 R:35.0000 rate:0.0700 aloss:

Episode:2596 meanR:44.1000 R:49.0000 rate:0.0980 aloss:0.2776 eloss:1.6039 aloss2:5.3758 exploreP:0.0100
Episode:2597 meanR:43.7500 R:31.0000 rate:0.0620 aloss:0.2426 eloss:1.5939 aloss2:5.3948 exploreP:0.0100
Episode:2598 meanR:43.7100 R:32.0000 rate:0.0640 aloss:0.2342 eloss:1.6594 aloss2:5.3971 exploreP:0.0100
Episode:2599 meanR:43.7200 R:40.0000 rate:0.0800 aloss:0.2466 eloss:1.6616 aloss2:5.3954 exploreP:0.0100
Episode:2600 meanR:43.9900 R:54.0000 rate:0.1080 aloss:0.2497 eloss:1.5849 aloss2:5.4026 exploreP:0.0100
Episode:2601 meanR:43.9600 R:46.0000 rate:0.0920 aloss:0.2504 eloss:1.5571 aloss2:5.4278 exploreP:0.0100
Episode:2602 meanR:43.6500 R:28.0000 rate:0.0560 aloss:0.2714 eloss:1.5888 aloss2:5.4435 exploreP:0.0100
Episode:2603 meanR:43.7200 R:32.0000 rate:0.0640 aloss:0.2477 eloss:1.6465 aloss2:5.4325 exploreP:0.0100
Episode:2604 meanR:43.8800 R:50.0000 rate:0.1000 aloss:0.2449 eloss:1.6500 aloss2:5.4225 exploreP:0.0100
Episode:2605 meanR:43.9500 R:65.0000 rate:0.1300 aloss:

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
